## Problem Definition

**The Task:**
- Based on the data from https://www.kaggle.com/wendykan/lending-club-loan-data, build a predictive model for loan defaults (i.e. learn to predict which loan payments will default) [Data to download](https://www.kaggle.com/wendykan/lending-club-loan-data/downloads/lending-club-loan-data.zip)

## Work Plan

Generally, **building any predictive model involves**:
- Understanding the business problems and challanges
- Exploring the available data sources, extracting and preparing data
- Defining how the models will learn
- Evaluating the models agaist the hold-out data
- Presenting and using predictive models


Here, we need to **be clear about the answers to some questions**.

What does "predicting loan defaults" mean for the business?
> Limiting the losses, and at the same time, not adversely impacting the revenue generated by giving loans.


What data sources are available and what are they like?
> We have one data source in this exercise.


How to define and learn predictive models?
> The algorithms need to be selected, which determines how the models will learn. Here, a range of algorithms will be tried to build Generalized Linear Models, Support Vector Machines, Random Forest Tress, Gradient Boosting Machines, Neural Nets, and others. Mainly, the scikit-learn library will be used. But others like xgboost, lightgbm, and keras will complement it

How to evaluate the predictive models?
> The available data must be split into train and test (or hold-out) sets. The first one will be used for model learning and tuning. The second one will be used at the end for the final evaluation of the model. It is crucial that the test set be hold out on the developing models untill the time of the final evaluation. The evaluation will be based on these measures: Accuracy, Precision, Recall, F1, ROC, PRC. However, in real life situations, more business oriented metrics, such as Expected Profit and Uncertainty (Risk) may be a better fit for the purpose

How to present and use predictive models?
> The models and their evaluation results must be presented so that people can make informed decisions. If beneficial, the models will be deployed to serve in real business conditions. Their performance need to be monitored to make sure that they adapt and perform well in ever changing conditions.

### Imports and Configuration Settings

In [1]:
# common imports and configuration settings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.max_columns = 80
pd.options.display.max_rows = 20

### See the data

In [2]:
# load the data
import sqlite3

with sqlite3.connect('lending-club-loan-data/database.sqlite') as db:
    df = pd.read_sql_query('SELECT * FROM loan', con=db)

# show the df's shape
df.shape

(887383, 75)

In [3]:
df.sample(1) # shows one row

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
373257,142598,18625317,20787974.0,6500.0,6500.0,6500.0,36 months,12.49%,217.42,B,B5,Physical therapist assistant,3 years,OWN,35000.0,Not Verified,Jun-2014,Current,n,https://www.lendingclub.com/browse/loanDetail....,None,other,Other,328xx,FL,6.34,1.0,Mar-1988,1.0,17.0,NaN,13.0,0.0,1210.0,2.1%,21.0,w,3371.5,3371.5,4130.98,4130.98,3128.5,1002.48,0.0,0.0,0.0,Jan-2016,217.42,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,None,0.0,0.0,4248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58200.0,NaN,NaN,NaN


In [4]:
# do some preliminary cleaning, i.e
# remove "index" which is generated by sqlite 
df.drop('index',axis=1, inplace=True)
# set "id" (a unique ID for the loan) as the index, which will be useful for investigations later on
df = df.set_index('id')

# keep only the rows where loan_amnt is not null (you cannot default on a null loan, can you?)
df = df[ df.loan_amnt.notnull() ]

# let's assume the default also occurs when loan_status is one of the following:
# - Charged off
# - Default
# - Does not meet the credit policy. Status: Charged Off
# - Late (31-120 days)
tmp_s = df.loan_status.replace( {
    'Charged Off': 'Default',
    'Late (31-120 days)': 'Default',
    'Does not meet the credit policy. Status:Charged Off': 'Default',
    'Does not meet the credit policy. Status:Fully Paid': 'Fully Paid',
})

### Assign the label (that the models need to learn to predict)

In [5]:
# set "DEFAULT" as the target variable; it can take on only either 0 or 1 
df.insert(0,'DEFAULT', np.where(tmp_s == 'Default', 1, 0)) # 1 - being the DEFAULT, 0 - good loan

# drop loan_status since its role has been taken by "DEFAULT"
df = df.drop('loan_status', axis=1)

### Split the data into train and test sets

In [6]:
# do the train-test split
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=.30)
df.shape, df_train.shape, df_test.shape

((887382, 73), (621167, 73), (266215, 73))

In [8]:
# verify that the split stratified the target variable (assigned label) equally
print('Train set target values incidence: \n', df_train.DEFAULT.value_counts()/df_train.shape[0] )
print('Test set target value incidence: \n', df_test.DEFAULT.value_counts()/df_test.shape[0] )

Train set target values incidence: 
 0    0.934142
1    0.065858
Name: DEFAULT, dtype: float64
Test set target value incidence: 
 0    0.932724
1    0.067276
Name: DEFAULT, dtype: float64


In [9]:
# save both sets
with sqlite3.connect('lending-club-loan-data/train.sqlite') as db:
    df_train.to_sql('loan', con=db, if_exists='replace', index=True, index_label='id')

with sqlite3.connect('lending-club-loan-data/test.sqlite') as db:
    df_test.to_sql('loan', con=db, if_exists='replace', index=True, index_label='id')


###### [Next: 2 data exploration and preparation](Predicting-Loan-Defaults-2-data-exploration-and-preparation.ipynb)